### How to generate a tensor or an array with interleaving elements from different sources

For example, given a a list of elements $[x_1, x_2, x_3, ... , x_n]$ , we want to generate a tensor $[sin(x_1), cos(x_1), sin(x_2), cos(x2), sin(x_3), cos(x_3), ... , sin(x_n), cos(x_n)]$ without going through the original list iteratively. 

We can achieve this goal easily with the following solution.

In [1]:
import torch

Create the original source sequence:

In [2]:
x = torch.arange(10) * 0.5 * torch.pi
print(x)
print(x.shape)

tensor([ 0.0000,  1.5708,  3.1416,  4.7124,  6.2832,  7.8540,  9.4248, 10.9956,
        12.5664, 14.1372])
torch.Size([10])


Then we unsqueeze it by adding an extra dimension at the last dimension.

In [3]:
x = torch.unsqueeze(x, dim=-1)
print(x)
print(x.shape)

tensor([[ 0.0000],
        [ 1.5708],
        [ 3.1416],
        [ 4.7124],
        [ 6.2832],
        [ 7.8540],
        [ 9.4248],
        [10.9956],
        [12.5664],
        [14.1372]])
torch.Size([10, 1])


Then we concatenates the sequences of tensors at the last dimension. 

In [4]:
x = torch.cat((torch.sin(x), torch.cos(x)), dim=-1)
print(x)
print(x.shape)

tensor([[ 0.0000e+00,  1.0000e+00],
        [ 1.0000e+00, -4.3711e-08],
        [-8.7423e-08, -1.0000e+00],
        [-1.0000e+00,  1.1925e-08],
        [ 1.7485e-07,  1.0000e+00],
        [ 1.0000e+00, -3.3777e-07],
        [-2.3850e-08, -1.0000e+00],
        [-1.0000e+00,  6.6361e-07],
        [ 3.4969e-07,  1.0000e+00],
        [ 1.0000e+00, -3.5775e-08]])
torch.Size([10, 2])


After that, we reshape the tensor to the desired shape by flattening the last two dimensions.

In [5]:
x = torch.flatten(x, start_dim=-2, end_dim=-1)
print(x)
print(x.shape)

tensor([ 0.0000e+00,  1.0000e+00,  1.0000e+00, -4.3711e-08, -8.7423e-08,
        -1.0000e+00, -1.0000e+00,  1.1925e-08,  1.7485e-07,  1.0000e+00,
         1.0000e+00, -3.3777e-07, -2.3850e-08, -1.0000e+00, -1.0000e+00,
         6.6361e-07,  3.4969e-07,  1.0000e+00,  1.0000e+00, -3.5775e-08])
torch.Size([20])


The same method can be applied to NumPy with some modifications.

In [6]:
import numpy as np

In [7]:
y = np.arange(10) * 0.5 * np.pi
print(y)
print(y.shape)

[ 0.          1.57079633  3.14159265  4.71238898  6.28318531  7.85398163
  9.42477796 10.99557429 12.56637061 14.13716694]
(10,)


However, sicne NumPy does not have `unsqeeze`, we have to use `expand_dims` instead.

In [8]:
y = np.expand_dims(y, axis=-1)
print(y)
print(y.shape)

[[ 0.        ]
 [ 1.57079633]
 [ 3.14159265]
 [ 4.71238898]
 [ 6.28318531]
 [ 7.85398163]
 [ 9.42477796]
 [10.99557429]
 [12.56637061]
 [14.13716694]]
(10, 1)


Concatenation is similar.

In [9]:
y = np.concatenate((np.sin(y), np.cos(y)), axis=-1)
print(y)
print(y.shape)

[[ 0.0000000e+00  1.0000000e+00]
 [ 1.0000000e+00  6.1232340e-17]
 [ 1.2246468e-16 -1.0000000e+00]
 [-1.0000000e+00 -1.8369702e-16]
 [-2.4492936e-16  1.0000000e+00]
 [ 1.0000000e+00  3.0616170e-16]
 [ 3.6739404e-16 -1.0000000e+00]
 [-1.0000000e+00 -4.2862638e-16]
 [-4.8985872e-16  1.0000000e+00]
 [ 1.0000000e+00  5.5109106e-16]]
(10, 2)


Finally, in NumPy we can utilize `reshape`.  Using `-1` as a parameter to reshape tells NumPy to infer the dimension there.

In [10]:
y = np.reshape(y, (*y.shape[:-2] ,-1))
print(y)
print(y.shape)

[ 0.0000000e+00  1.0000000e+00  1.0000000e+00  6.1232340e-17
  1.2246468e-16 -1.0000000e+00 -1.0000000e+00 -1.8369702e-16
 -2.4492936e-16  1.0000000e+00  1.0000000e+00  3.0616170e-16
  3.6739404e-16 -1.0000000e+00 -1.0000000e+00 -4.2862638e-16
 -4.8985872e-16  1.0000000e+00  1.0000000e+00  5.5109106e-16]
(20,)


They can both support N dimensional sequences as well.

In [11]:
a = torch.arange(10).reshape(1 , 2 , 1, 5) * 0.5 * torch.pi
print(a)
print(a.shape)

tensor([[[[ 0.0000,  1.5708,  3.1416,  4.7124,  6.2832]],

         [[ 7.8540,  9.4248, 10.9956, 12.5664, 14.1372]]]])
torch.Size([1, 2, 1, 5])


In [12]:
a = torch.unsqueeze(a, dim=-1)
a = torch.cat((torch.sin(a), torch.cos(a)), -1)
a = torch.flatten(a, -2, -1)
print(a)
print(a.shape)

tensor([[[[ 0.0000e+00,  1.0000e+00,  1.0000e+00, -4.3711e-08, -8.7423e-08,
           -1.0000e+00, -1.0000e+00,  1.1925e-08,  1.7485e-07,  1.0000e+00]],

         [[ 1.0000e+00, -3.3777e-07, -2.3850e-08, -1.0000e+00, -1.0000e+00,
            6.6361e-07,  3.4969e-07,  1.0000e+00,  1.0000e+00, -3.5775e-08]]]])
torch.Size([1, 2, 1, 10])


In [13]:
b = np.arange(10).reshape(1, 2 , 1, 5) * 0.5 * np.pi
print(b)
print(b.shape)

[[[[ 0.          1.57079633  3.14159265  4.71238898  6.28318531]]

  [[ 7.85398163  9.42477796 10.99557429 12.56637061 14.13716694]]]]
(1, 2, 1, 5)


In [14]:
b = np.expand_dims(b, -1)
b = np.concatenate((np.sin(b), np.cos(b)), axis=-1)
b = np.reshape(b, (*b.shape[:-2] ,-1))
print(b)
print(b.shape)

[[[[ 0.0000000e+00  1.0000000e+00  1.0000000e+00  6.1232340e-17
     1.2246468e-16 -1.0000000e+00 -1.0000000e+00 -1.8369702e-16
    -2.4492936e-16  1.0000000e+00]]

  [[ 1.0000000e+00  3.0616170e-16  3.6739404e-16 -1.0000000e+00
    -1.0000000e+00 -4.2862638e-16 -4.8985872e-16  1.0000000e+00
     1.0000000e+00  5.5109106e-16]]]]
(1, 2, 1, 10)
